# Helper functions for PlaineMorte.jl

In [1]:
using NBInclude
#Call the different paths needed
@nbinclude("paths_download.ipynb");

In [2]:
#Constant definition
g = 9.81
rhow = 1000.0;

In [ ]:
using LsqFit # a package for least square fitting
using MonteCarloMeasurements

"""
    curve_fit_MCMeasurements(model, x, y, p0)

Does LsqFit.curve_fit with MonteCarloMeasurements Vectors.  Same input as `curve_fit`.
"""
function curve_fit_MCMeasurements(model, x, y, p0)
    if eltype(x)<:Particles && eltype(y)<:Particles
        len = length(x[1].particles)
        pout = [eltype(p0)[] for p in p0]
        for i=1:len
            p = curve_fit(model, [xx.particles[i] for xx in x], [yy.particles[i] for yy in y], p0)
            @assert p.converged
            for j=1:length(p0)
                push!(pout[j], p.param[j])
            end
        end
    elseif eltype(y)<:Particles
        len = length(y[1].particles)
        pout = [eltype(p0)[] for p in p0]
        for i=1:len
            p = curve_fit(model, x, [yy.particles[i] for yy in y], p0)
            @assert p.converged "$i"
            for j=1:length(p0)
                push!(pout[j], p.param[j])
            end
        end
    elseif eltype(x)<:Particles
        len = length(x[1].particles)
        pout = [eltype(p0)[] for p in p0]
        for i=1:len
            p = curve_fit(model, [xx.particles[i] for xx in x], y, p0)
            @assert p.converged
            for j=1:length(p0)
                push!(pout[j], p.param[j])
            end
        end
    else
        error()
    end
    return [Particles(po) for po in pout]
end

In [3]:
"""
    fit_calibration(bucketsize, solution, calis...)

Fits a line of best fit through the calibration data going through the origin.  
Returns the function of this line: `f(cond-cond_at_0) -> concentration`.

Also prints the parameters values +/- 95% confidence intervals.

Uses the package LsqFit: https://github.com/JuliaOpt/LsqFit.jl
"""
function fit_calibration(bucketsize, solution, calis...)
    # subtract readout at zero concentration
    for c in calis
        @assert c[1,1]==0 "First row needs to be zero reading!"
        c[:,2] .= c[:,2].-c[1,2]
    end
    
    #first compute ml_to_concentration : both bucketsize and solution has n values for n element in cali

    #then only concatenate all concentration
    #conc=vcat(c1,c2..)
    
    # concatenate all calibrations
    
    cali = vcat(calis...)
    conc = ml_to_concentration(cali[:,1], solution, bucketsize)
    delta_readout = cali[:,2]
    # Fit line using https://github.com/JuliaOpt/LsqFit.jl
    fn(delta_readout, p) = p[1]*delta_readout # p[1]==a, p[2]==b
    para_weights = [0.5] # equal weights to parameters
    fit = curve_fit(fn, delta_readout, conc, para_weights)
    errors = margin_error(fit, 1-0.95)
    println("""
    Estimated linear fit: f(delta_cond) = a*conc with (margin errors 5%)
     a = $(round(fit.param[1],sigdigits=3))±$(round(errors[1],sigdigits=3))
    """)
    return (delta_readout) -> fn(delta_readout, fit.param)
    
    
end;

"""
    fit_calibration_LAB_2000(calis...)
ONLY for LAB experiment conducted the 9th October, using the 0-2000 μS/cm range. Because of the unique protocol 
of dilution (Both Solution 1g/l and 10g/l were used).
Fits a line of best fit through the calibration data going through the origin.  
Returns the function of this line: `f(cond-cond_at_0) -> concentration`.

Also prints the parameters values +/- 95% confidence intervals.

Uses the package LsqFit: https://github.com/JuliaOpt/LsqFit.jl
"""

function fit_calibration_LAB_2000(calis...)
    # subtract readout at zero concentration
    for c in calis
        @assert c[1,1]==0 "First row needs to be zero reading!"
        c[:,2] .= c[:,2].-c[1,2]
    end
    cali = vcat(calis...)
    conc1 = ml_to_concentration(cali[1:10,1], solution1, bucketsize2);
    conc2 = ml_to_concentration(cali[11:19,1], solution2, bucketsize2) .+ conc1[end]
    conc= vcat(conc1,conc2)
    delta_readout = cali[:,2] #conductivity

    # Fit line using https://github.com/JuliaOpt/LsqFit.jl
    fn(delta_readout, p) = p[1]*delta_readout.^2 + p[2]*delta_readout;  # p[1]==a, p[2]==b  
    para_weights = [0.5, 0.5] # equal weights to parameters
    fit = curve_fit(fn, delta_readout, conc, para_weights)
    errors = margin_error(fit, 1-0.95)
    #standard error
    sigma = stderror(fit)  #size of fit.param
    
    println("""
    Estimated fit: f(delta_cond) = a*conc^2 + b*conc with (± std dev)
     a = $(round(fit.param[1],sigdigits=3))±$(round(sigma[1],sigdigits=3))
        and b = $(round(fit.param[2],sigdigits=3))±$(round(sigma[2],sigdigits=3))
    """)
    #I write again the function fitted but with errors associated, to calculate conc ± σ from cond ± σ
    
    cond2conc(delta_readout, p, err) =(p[1] ± err[1])*delta_readout.^2 + (p[2] ± err[2])*delta_readout
    
    
    return (delta_readout) -> cond2conc(delta_readout ± 0, fit.param, sigma) #Unstable for little conductivity...
    
end;

"""
    fit_calibration_LAB_200(calis...)

Similary to fit_calibration_2000 but in the 0-2000 μS/cm range, it is specialy made for the 10th october calibration.
"""

function fit_calibration_LAB_200(calis...)
    # subtract readout at zero concentration
    for c in calis
        @assert c[1,1]==0 "First row needs to be zero reading!"
        c[:,2] .= c[:,2].-c[1,2]
    end
    cali = vcat(calis...)
    
    conc3 = ml_to_concentration(cali[1:10,1], solution1, bucketsize2);
    conc4 = ml_to_concentration(cali[11:20,1], solution2, bucketsize2) .+ conc3[end]
    conc= vcat(conc3,conc4)
    delta_readout = cali[:,2]

    # Fit line using https://github.com/JuliaOpt/LsqFit.jl
    fn(delta_readout, p) = p[1]*delta_readout 
    para_weights = [0.5] # equal weights to parameters
    fit = curve_fit(fn, delta_readout, conc, para_weights)
    errors = margin_error(fit, 1-0.95)
    sigma = stderror(fit)
    println("""
    Estimated fit: f(delta_cond) = a*conc with (± std dev)
     a = $(round(fit.param[1],sigdigits=3))±$(round(sigma[1],sigdigits=3)) 
    """)
    return (delta_readout) -> fn(delta_readout, fit.param)
end;

In [4]:
using Dates
using DelimitedFiles: readdlm # Date time handling; CSV file handling

"""
    read_salt_injections(filename)
    To read all the salt injection (location, time, mass injected...) from the 
appropriate CSV file

"""


function read_salt_injections(filename)
    if !isfile(filename)
        error("Filename $filename is not a file!")
    end
    d, head = readdlm(filename, ';', header=true)
    out = Dict{Symbol,Any}() 
    # Numero injection
    out[:num] = d[:,1]
    # Date
    fmtd = "d/m/y"
    out[:d] = [Date(dd, fmtd) for dd in d[:,2]]
    #Time
    fmtt = "H:M:S"
    out[:t] = [Time(dd, fmtt) for dd in d[:,3]]
    # location
    out[:inj] = d[:,4]
    # Mass
    out[:mass] = d[:,5]
    return out
end

read_salt_injections (generic function with 1 method)

In [3]:
path_CTD

"C:\\Users\\christophe\\Desktop\\home\\GitHub\\PlaineMorteLake_hydro_thermodynamics\\notebooks\\../data/CTD/"

In [5]:
#Air Pressure Reccord in summer 2019 from Fribourg station on Plaine Morte (2690 m)
using Dates
using DelimitedFiles: readdlm 
p, head = readdlm(joinpath(path_CTD, "PLM_atmo_pressure/PLM_atmo_pressure_2019.csv"), ';', header=true)
    p_atm = Dict{Symbol,Any}() 
    # DateTime
    fmtdt = "d.m.y H:M:S"
    p_atm[:t] = [DateTime(pp, fmtdt)+Dates.Hour(2) for pp in p[:,1]];  #UTM to UTC conversion
    # Air pressure
    p_atm[:press] = p[:,2];

#Air Pressure Reccord in summer 2020 from Fribourg station on Plaine Morte (2690 m)
p, head = readdlm(joinpath(path_CTD, "PLM_atmo_pressure/PLM_atmo_pressure_2020.csv"), ';', header=true)
    p_atm_2020 = Dict{Symbol,Any}() 
    # DateTime
    fmtdt = "d.m.y H:M:S"
    p_atm_2020[:t] = [DateTime(pp, fmtdt) for pp in p[:,1]];  #raw data already in ETC Time
    # Air pressure
    p_atm_2020[:press] = p[:,2];

In [6]:
using Dates
using DelimitedFiles: readdlm # Date time handling; CSV file handling

# These will read files containing time series of a combination of the following sensors:
# conductivity, pressure, temperature
# The convention is to return a dict with keys as appropriate:
# :t [date-time stamp], :cond [μS/cm], :temp [C], :press [m H2O]
#
# (Note, the funny ":" in front of the name makes a special kind of string, a so-called Symbol)

"""
    read_WTW(filename)

This function reads a file from the WTW conductivity sensor and
returns:

Dict with keys: :t [date-time stamp], :cond [μS/cm], :temp [C]

Note, that the input file usually contains several traces.  Split them up with 
`split_conductivity_data`.
"""
function read_WTW(filename)
    if !isfile(filename)
        error("Filename $filename is not a file!")
    end
    if !startswith(splitdir(filename)[2], "AD")
        warn("Read in a file starting with `AD`.  (The `AC` files use a comma for the decimal point.")
    end
    d, head = readdlm(filename, ';', header=true)
    out = Dict{Symbol,Any}() # key has the be Symbol, value can be anything
    # time 12.08.2016 13:36:58
    fmt = "d.m.y H:M:S"
    out[:t] = [DateTime(dd, fmt) for dd in d[:,4]]
    # conductivity
    out[:cond] = d[:,5]
    units = d[:,6]
    @assert all(units.=="\xb5S/cm") "Units not in μS/cm!"
    # temp
    out[:temp] = d[:,8]
    
    # purge any records which are simultaneous (not sure why this happens with the WTW)
    purge = findall(diff(out[:t]).==Second(0))
    for p in reverse(purge)
        deleteat!(out[:t], p)
        deleteat!(out[:cond],p)
        deleteat!(out[:temp], p)
    end

    return out
end


using DelimitedFiles, Dates


#temperature calibration datas (°C) in lab from Temperature_calibration
sensor_T = Dict() 
sensor_T[205144]=-0.104
sensor_T[205145]=0.076
sensor_T[205309]=-0.021
sensor_T[207265]=-0.319
sensor_TOB1 = Dict() 
sensor_TOB1[205144]=-0.123
sensor_TOB1[205145]=0.002
sensor_TOB1[205309]=0.018
sensor_TOB1[207265]=0.161

"""
         read_Keller(filename;
                     presshead="P1",
                     condhead="ConRaw",
                     temphead="TOB1",
                     skipstart=8,
                     )

Reads a Keller pressure/CTD sensor.  However, you probably want to use
- `read_Keller_DCX22_CTD`, 
- `read_Keller_DCX22` and 
- `read_Keller_DC22` 

Returns a dict with keys as appropriate:
- :t [date-time stamp]
- :cond [μS/cm]
- :temp [C]
- :press [m H2O]
"""
function read_Keller(filename;
                     presshead="P1",
                     condhead="ConRaw",
                     temphead="TOB1",
                     temphead2="T",
                     skipstart=8,
                     )
    
    #extract sensor serial number
    s = filename[length(path_CTD)+12:length(path_CTD)+17]
    #We adapt the filename according to the user (difference in paths)
    #for christophe local machine s=filename[99:104]
    #for vaw new computer s=filename[92:97]
    sensor=parse(Int, s)
    
    #read the CSV
    d,h = readdlm(filename, ';', skipstart=skipstart, header=true)
    h = h[:] # h is a 1x2 matrix, change to a vector

    out = Dict{Symbol,Any}()
    # find date-time rows
    id, it = findfirst(h.=="Date"), findfirst(h.=="Time")
    # time 12.08.2016 13:36:58
    fmtd, fmtt = "d/m/y", "H:M:S"
    out[:t] = [Date(dd, fmtd) + Time(tt, fmtt) for (dd,tt) in zip(d[:,id], d[:,it])]

    #
    for (head, key) in [(presshead, :press),
                        (condhead, :cond),
                        (temphead, :temp),
                       (temphead2, :temp2)]
        i = findfirst(h.==head) # see if there is one
        tmp = Float64[]
        if i!=nothing
            out[key] = [s=="" ? missing :
                        s isa AbstractString ? parse(Float64, replace(s, ","=>".")) : Float64(s) for s in d[:,i]]
            # convert mS/cm to μS/cm
            if key==:cond
                out[:cond] = out[:cond].*1000
            end
            #correct temperature for each sensor
            if key==:temp
                out[:temp]=out[:temp].-sensor_TOB1[sensor]
            end
            if key==:temp2
                out[:temp2]=out[:temp2].-sensor_T[sensor]
            end
            
            # convert pressure from mbar to m H2O
            if key==:press 
                if out[:t][1]> DateTime(2019,9,5)
                   out[:press] = out[:press]  #No atm correction availbale after 4th September
                else
                    j=1 
                    while p_atm[:t][j] <=out[:t][1]   #start of air pressure record
                    j=j+1
                     end       #p_atm[:t][j] is then the start ot air pressure time serie matching the :press time serie
                     
                     for i in 1:length(out[:t])
                        if out[:t][i]<=p_atm[:t][j]  
                            out[:press][i] = ((out[:press][i].-p_atm[:press][j]).*1e2./(g*rhow)).+0.05 #5cm correction from data readings (Δz=39m ?)

                        else
                            j=j+1
                            out[:press][i] = ((out[:press][i].-p_atm[:press][j]).*1e2./(g*rhow)).+0.05
                        
                        end
                 
                     end
                end
              
                end    
                    
             end 
        end
      
    
     # purge any records which are simultaneous (not sure why this happens with the WTW)
    purge = findall(diff(out[:t]).==Second(0))
    for p in reverse(purge)
        deleteat!(out[:t], p)
        deleteat!(out[:cond],p)
        deleteat!(out[:temp], p)
        deleteat!(out[:temp2], p)
        deleteat!(out[:press], p)
    end
 
    # check lengths and remove all "missing"
    l = length(out[:t])
    topurge = []
    for v in values(out)
        @assert length(v)==l
        append!(topurge, findall(v.===missing))
    end
    topurge = sort(unique(topurge))
    for (k,v) in out
        deleteat!(v, topurge)
        if k!=:t
            out[k] = Float64.(v) # make the vector an
        end
    end
    return out
end

read_Keller_DCX22_CTD(filename) = read_Keller(filename)  #Actually, read_Keller function is already adapted to CTD

#A dedicated function to read Keller DCX-22
function read_Keller_DCX22(filename;
                            presshead="P1",
                            temphead="TOB1",
                            skipstart=8)
    if !isfile(filename)
        error("Filename $filename is not a file!")
    end                                   
    
    #read the CSV
    d,h = readdlm(filename, ';', skipstart=skipstart, header=true)
    h = h[:] # h is a 1x2 matrix, change to a vector

    out = Dict{Symbol,Any}()
    # find date-time rows
    id, it = findfirst(h.=="Date"), findfirst(h.=="Time")
    # time 12.08.2016 13:36:58
    fmtd, fmtt = "d.m.y", "H:M:S"
    out[:t] = [Date(dd, fmtd) + Time(tt, fmtt) for (dd,tt) in zip(d[:,id], d[:,it])]
    
    for (head, key) in [(presshead, :press),
                        (temphead, :temp)]
        i = findfirst(h.==head) # see if there is one
        tmp = Float64[]
        if i!=nothing
            out[key] = [s=="" ? missing :
                    s isa AbstractString ? parse(Float64, replace(s, ","=>".")) : Float64(s) for s in d[:,i]]
         
            
            if key==:press 
                # convert pressure from mbar to m H2O
                #correction available for 1 August -> 1 September 2020 (Plaine Morte Fribourg station)
                 
                    j=1 
                    while p_atm_2020[:t][j] <=out[:t][1]   #start of air pressure record
                    j=j+1
                    end       #p_atm_2020[:t][j] is then the start ot air pressure time serie matching the :press time serie
                     
                     for i in 1:length(out[:t])
                        if out[:t][i]<=p_atm_2020[:t][j]  
                            out[:press][i] = ((out[:press][i].-p_atm_2020[:press][j]).*1e2./(g*rhow)) 

                        else
                            j=j+1
                            out[:press][i] = ((out[:press][i].-p_atm_2020[:press][j]).*1e2./(g*rhow)) 
                        
                        end
                      
            
                end     
            end 
        
        end
    end   
    
     # purge any records which are simultaneous (not sure why this happens with the WTW)
    purge = findall(diff(out[:t]).==Second(0))
    for p in reverse(purge)
        deleteat!(out[:t], p)
        deleteat!(out[:cond],p)
        deleteat!(out[:temp], p)
        deleteat!(out[:temp2], p)
        deleteat!(out[:press], p)
    end
 
    # check lengths and remove all "missing"
    l = length(out[:t])
    topurge = []
    for v in values(out)
        @assert length(v)==l
        append!(topurge, findall(v.===missing))
    end
    topurge = sort(unique(topurge))
    for (k,v) in out
        deleteat!(v, topurge)
        if k!=:t
            out[k] = Float64.(v) # make the vector an
        end
    end
    return out
end


read_Keller_DC22(filename) = read_Keller(filename,
                                         presshead="p1/mbar",
                                         temphead="t1/\xb0C",
                                         skipstart=24
                                         )

read_Keller_DC22 (generic function with 1 method)

In [7]:
#using PyPlot

"""
   plotit(filename, sensor, variable)

Plots the results from one file. 
"""
function plotit(filename, sensor, variable=:cond ; plottime=true)
    d = read_Keller_DCX22_CTD(filename)
    # plot
    fig=figure()
    grid(true)
    title("$filename, sensor=$sensor, variable=$variable")
    xlabel("Time")
    if variable==:cond
        ylabel("Conductivity (μS//cm)")
        color=:blue
        elseif variable==:press
        ylabel("Water depth (m)")
        color=:blue
        else variable==:temp
        ylabel("Temperature (°C)")
        ylim(-0.1,5)
        color=:red
    end  
    if plottime
        plot(d[:t],d[variable],linestyle="-",color,marker="None")
        fig[:autofmt_xdate](bottom=0.2,rotation=30,ha="right")
    else
        plot(d[variable])
    end
end

plotit

In [8]:
"""
    split_conductivity_data_indices(d, indices::Vector)

Splits the CTD time series into separate traces according to passed in indices.

Example:

    traces = split_conductivity_data_indices(d, [500:3400, 5000:6789])
"""
function split_conductivity_data_indices(d, indices::Vector)
    traces = []
    t = d[:t]
    for index in indices
        tmp = Dict{Symbol,Any}()
        tmp[:t] = convert(Vector{Float64}, Dates.value.(t[index]-t[index[1]])/1000)
        tmp[:tstart] = t[index[1]]
        tmp[:cond] = d[:cond][index]
        if haskey(d, :temp)
            tmp[:temp] = d[:temp][index]
        end
        if haskey(d, :press)
            tmp[:press] = d[:press][index]
        end
        push!(traces, tmp)
    end
    
    return traces
end
    


"""
    split_conductivity_data_timegap(outdict, timegap=5.0, mincond=0.5, maxtemp=2)

Splits up the time series returned by `read_conductivity_data` into individual
traces, mostly by using that there is a big time-gap in the record.

First the datapoints which have `outdict[:cond]<mincond` or `outdict[:temp]>maxtemp` are deleted.
Then, the split is done where the time-gap is bigger than `timegap` seconds.

The output is a list of traces:

`traces = [trace1, trace2, etc]`

where `trace1` contains the time series with `:t` in seconds after recoding started, `:tstart` is 
the date-time of the start of the recording.  You should also add the injection `:mass` to each trace 
manually.

Thus to access the times of the second trace do `traces[2][:t]`, and to get the corresponding
conductivity readout `traces[2][:cond]`.
"""
function split_conductivity_data_timegap(out; timegap=5.0, mincond=0.0, maxtemp=2, minpeak=6, minduration=60)
    # delete where cond<mincond and temp>maxtemp
    out = deepcopy(out)
    tokeep = (out[:cond].>mincond) .& (out[:temp].<maxtemp)
    for (k,v) in out
        out[k] = v[tokeep]
    end

    t = out[:t]
    cond = out[:cond]   

    # split it up according to time gaps
    splits_at = findall(diff(t).>Second(timegap))
    push!(splits_at, length(t))
    traces = []
    i = 1
    for sp in splits_at
        tt = convert(Vector{Float64}, Dates.value.(t[i:sp]-t[i])/1000)
        dt = t[i+1]-t[i]
        if !all(diff(t[i:sp]).==dt) 
            #@warn "Logging intervals are not evenly spaced!"
        end
        tmp = Dict{Symbol,Any}()
        tmp[:t] = tt
        if tt[end]-tt[1]<minduration
            continue
        end
        tmp[:tstart] = t[i]
        tmp[:cond] = cond[i:sp]
        if maximum(tmp[:cond])<minpeak
            continue
        end
        if haskey(out, :temp)
            tmp[:temp] = out[:temp][i:sp]
        end
        if haskey(out, :press)
            tmp[:press] = out[:press][i:sp]
        end
        push!(traces, tmp)
        i = sp+1
    end
    
    return traces
end

"""
     split_conductivity_data(out;
                             slice_cond=2.5,  # peaks are selected where cond is above this value
                             slice_padding_start=20, # add a padding of in front of detected peak
                             slice_padding_end=3*60, # add a padding of in back of detected peak
                             mincond=0.5, maxtemp=3, # also drop points which have conductivity below mincond and above maxtemp
                             smooth_window=10) # smoothing window used in peak detection

Splits up the time series returned by `read_conductivity_data` into individual
traces, mostly by heuristics around where peaks are.  A bit hit and miss.

The output is a list of traces:

`traces = [trace1, trace2, etc]`

where `trace1` contains the time series with `:t` in seconds after recoding started, `:tstart` is
the date-time of the start of the recording.  You should also add the injection `:mass` to each trace
manually.

Thus to access the times of the second trace do `traces[2][:t]`, and to get the corresponding
conductivity readout `traces[2][:cond]`.
"""
function split_conductivity_data(out;
                                 mincond=0.5, maxtemp=3,
                                 slice_cond=2.5,
                                 slice_padding_start=20,
                                 slice_padding_end=3*60,
                                 smooth_window=10)
    out = deepcopy(out)

    # smooth cond with running mean
    cond = out[:cond]
    t = out[:t]
    condsmooth = zeros(length(cond))
    for i=1:length(t)
        ## running mean
        tmp = 0.0
        n = 0
        for j=max(1,i-smooth_window):min(i+smooth_window, length(t))
            tmp += cond[j]
            n += 1
        end
        condsmooth[i] = tmp/n

        # ## median
        # condsmooth[i] = median(cond[max(1,i-smooth_window):min(i+smooth_window, length(t))])
    end

    # locate peaks with slice_cond and cut, also cut if time-gap bigger than timegap
    inds = []

    tokeep = condsmooth.>slice_cond
    # add +/- slice_inds
    s = 1
    while true
        s = findnext(tokeep, s)
        if s==nothing
            break
        end
        e = findnext(.!tokeep, s)
        push!(inds, max(1, s-slice_padding_start):min(e+slice_padding_end, length(tokeep)))
        s = e+1
        if s>length(tokeep)
            break
        end
    end

    traces = []
    i = 1
    for ind in inds
        tt = convert(Vector{Float64}, Dates.value.(t[ind]-t[ind[1]])/1000)
        tmp = Dict{Symbol,Any}()

        # delete where cond<mincond and temp>maxtemp
        tokeep = (out[:cond][ind].>mincond) .& (out[:temp][ind].<maxtemp)

        tmp[:t] = tt[tokeep]
        tmp[:tstart] = t[ind][tokeep][1]
        tmp[:cond] = cond[ind][tokeep]
        tmp[:condsmooth] = condsmooth[ind][tokeep]
        push!(traces, tmp)
    end
    return traces
end
;

In [9]:
"""
    integrate_concentration(t, conc, t1=0.0, t2=Inf)

Integrates the concentration time series.

Input:
- t -- times (s)
- t1, t2 -- integrate from t1 to t2.  Note that the concentration should be 
            close to zero at both t1 and t2 as we want to integrate over the whole curve.
- conc -- concentration (g/l) time series (convert conductivity with f_readout2conc 
          to a concentration)

Output:

- integrated concentration (g s/l) == (kg s/ m^3)
"""
function integrate_concentration(t, conc, t1=0.0, t2=Inf)
    inds = findfirst(t.>=t1):findlast(t.<=t2)
    dt = t[2]-t[1]
    out = sum(conc[inds]*dt) # approximate the integral by a sum
    return out
end
    
"""
    calcQ(trace, t1=0.0, t2=Inf, delta_cond2conc=delta_cond2conc)

Calculate discharge from sensor readout.

Input:
- trace -- time series of sensor readout, including a field `:mass` with mass of salt injected

Optional:
- t1,t2 -- integration limits (otherwise the whole series is integrated)
- delta_readout2conc -- if you got several `delta_readout2conc` functions, then
                        pass it in explicitly.

Output:
- discharge Q (m^3/s)
"""
function calcQ(trace, t1=0.0, t2=Inf, delta_cond2conc=delta_cond2conc)
    t, cond = trace[:t], trace[:cond]
    i1, i2 = findfirst(t.>=t1), findlast(t.<=t2)
    delta_cond = cond .- mean(cond[[i1,i2]])  #normalize from the actual conductivity background
    for i in eachindex(delta_cond)
        if delta_cond[i] < 0
            delta_cond[i] = 0   #because negative cond lead to bug in exponent fit (complex number)
        end
    end
    conc = delta_cond2conc(delta_cond)  #we take mean becasue conc is a particle 

    #uncertainty on mass = 1g = 0.001kg

    return (trace[:mass] ± 0.001)/integrate_concentration(t, conc, t1, t2)
end;

In [10]:
"""
    boxcar(A::AbstractArray, window, [, weights, keepmask])
    boxcar(A::AbstractArray, windows::Tuple, [, weights, keepmask])
    boxcar(A::AbstractArray, window::AbstractArray, [, weights, keepmask])
Boxcar filter.  The two argument call skips NaNs.  The three & four
argument call uses weights and propagates NaNs, it can be a lot faster.
Smoothing occurs over +/-window indices, 0 corresponds to no smoothing.
The window can be specified as:
- integer, for a symmetric window in all dimensions
- a tuple to give lower and upper windows
- a tuple of tuples to give different lower and upper windows for all dimensions
- a array of size(A) for a different, symmetric window at each point.
Weights, if given, will use those relative weights for averaging.  Note that
points which have value==NaN and weight==0 will not poison the result.
No average is calculated for points where keepmask==true, instead
their original value will be kept.
Notes:
- For the weights it may be faster to use non-Bool arrays nor BitArrays,
  say Int8.  Note that NaNs where weight==0 will not poison the result.
- Also works for Vectors.
From http://julialang.org/blog/2016/02/iteration
"""
boxcar(A::AbstractArray, window) = boxcar(A, (window,window))
function boxcar(A::AbstractArray, windows::Tuple)
    window_lower, window_upper = windows
    out = similar(A)
    R = CartesianIndices(size(A))
    I1, Iend = first(R), last(R)
    I_l = CartesianIndex(I1.I.*window_lower)
    I_u = CartesianIndex(I1.I.*window_upper)
    for I in R # @inbounds does not help
        out[I] = NaN
        n, s = 0, zero(eltype(out))
        for J in CartesianIndices(UnitRange.(max(I1, I-I_l).I , min(Iend, I+I_u).I) ) # used to be
        # for J in max(I1, I-I_l):min(Iend, I+I_u) ## in Julia 1.1
            if !ismissing(A[J]) && !isnan(A[J])
                s += A[J]
                n += 1
            end
        end
        out[I] = s/n
    end
    out
end

boxcar (generic function with 2 methods)

In [11]:
"""
function smooth_d(d; windows)

Smooth the time serie d (returned by read_keller) according to the windows asked. 
By default the smooth window of pressure time series is 10, for temperature (T) it is 30.

"""
function smooth_d(d; windows=Dict(:press=>10,
                                  #:temp=>10,  #if add temp, make a test to find it : !isnothing
                                  :temp2=>30))
    out = Dict()
    for (k,v) in windows
        out[k] = boxcar(d[k], v)
    end
    return out
end

smooth_d

In [12]:
using Statistics
"""
function upscale_timeseries(temp, t, period="period")

Up scale the time series : daily and hourly. The function returns the time serie averaged on each interval
specified (day or hours).
The mean for daily scale is associated at noon of the same day.

The mean for hourly scale is associated at the beginnng of the hour.

Achtung when there is NaN the mean dosn't take it, thus we should check if there is enough value in the time period
considered to make the mean relevant

"""


function upscale_timeseries(A, t, period="period")
    
    if period == "day"

A_daily = Float64[]
A_daily_mean = []
time_day = []

d = day(t[1])
mon = month(t[1])

for i in 1:length(A)

    if day(t[i]) == d && i !== length(A)
        if !isnan(A[i])   
        push!(A_daily, A[i])     
        end
                
    elseif i == length(A) #To record daily mean even if the day is not complete - 
                                         #I made it to compute daily Q_lake until 27th July
        avg = mean(A_daily)
        push!(A_daily_mean, avg)
        push!(time_day, DateTime(2019,mon,d))
        empty!(A_daily)
              

    elseif day(t[i]) !== d && month(t[i]) == mon
        
        avg = mean(A_daily)
        push!(A_daily_mean, avg)
        push!(time_day, DateTime(2019,mon,d))
        empty!(A_daily)
        d = d + 1
                
        
    elseif day(t[i]) !== d && month(t[i]) !== mon
        
        avg = mean(A_daily)
        push!(A_daily_mean, avg)
        push!(time_day, DateTime(2019,mon,d))
        empty!(A_daily)
        d = 1
        mon = mon + 1
    
    end
end
    
    return (time_day .+ Hour(12),A_daily_mean);
    
elseif period == "hour"
        
        A_hour = Float64[]
        A_hour_mean = []
        time_hour = []

h = hour(t[1])
d = day(t[1])
mon = month(t[1])

for i in 1:length(A)

    if hour(t[i]) == h && day(t[i]) == d && month(t[i]) == mon
        if !isnan(A[i])   
        push!(A_hour, A[i])
            
        end
    
    elseif hour(t[i]) !== h && day(t[i]) == d && month(t[i]) == mon
        
        avg = mean(A_hour)
        push!(A_hour_mean, avg)
        push!(time_hour, DateTime(2019,mon,d,h))
        empty!(A_hour)
        h = h + 1
        
    elseif hour(t[i]) !== h && day(t[i]) !== d && month(t[i]) == mon
        
        avg = mean(A_hour)
        push!(A_hour_mean, avg)
        push!(time_hour, DateTime(2019,mon,d,h))
        empty!(A_hour)
        h = 0        
        d = d + 1
                
    elseif hour(t[i]) !== h && day(t[i]) !== d && month(t[i]) !== mon
        
        avg = mean(A_hour)
        push!(A_hour_mean, avg)
        push!(time_hour, DateTime(2019,mon,d,h))
        empty!(A_hour)
        h = 0        
        d = 1
        mon = mon + 1
                
    end
end
    
    return (time_hour,A_hour_mean);
        
end
    
end

upscale_timeseries (generic function with 2 methods)

In [13]:
# function to convert the DateTime in decimal to interpolation x axis -- Start the 9th July --> End August
# Temporal resolution : minute
using Dates

function datetime2decimal(t::DateTime)
    if Dates.month(t) == 7     
        d = Dates.day(t) - 8  ## start 9th July ##
        
    elseif Dates.month(t) == 8
        d = Dates.day(t) + 31 - 8 #31 days in July
        
    elseif Dates.month(t) == 9  
        d = Dates.day(t) + 31 - 8 + 31 # 31th days in Aungust    
    end
    h = Dates.hour(t)    
    m = Dates.minute(t)   
    time_dec = d + h/24 + m/(24*60);
    end;

In [2]:
"""
function isnan_part(x) 

(part for particles)

Return false is x is a particle, true if there is a NaN or Inf
Remark: isnan(NaN ± Inf) is false, isnan_part(x) is true

Note: this automatically throw an error if there is a NaN (instead of a Inf) behind the ±
Todo: catch this error too
"""
function isnan_part(x)
    
try (x).particles
    catch e
    throw(e) #throw error if x is not a particle
end
    if isnan(mean(x)) || isnan(std(x))
        return true
    else
        return false
    end
    
end

isnan_part

In [ ]:
""" 
reynolds(h, w, q)

input :
-- h water stage (m)
-- w width (m)
-- q discharge (m3/s)

h and q have the same dimension

Assuming a rectangular cross-section

output:
--reynolds number at one cross section

"""
function reynolds(h, w, q)
    η = 1.8E-6; #kinematic viscosity of water
    Pw = @. w + 2*h # wetted perimeter: rectangular cross section
    A = @. w*h
    return @. (q*4)/(η*Pw)
end


""" 
stream_veloctiy(h, w, q)

input :
-- h water stage (m)
-- w width (m)
-- q discharge (m3/s)

h and q have the same dimension

Assuming a rectangular cross-section

output:
--The averaged stream velocity at the cross section

"""
function stream_velocity(h, w, q)
    A = @. w*h # rectangular wetted cross section
    return @. q/A  #the latter is the velocity associated to reynolds calculation
end